In [ ]:
import os
import numpy as np
from PIL import Image
import random
import math
import pandas as pd
import cv2
import pandas as pd
from tqdm import tqdm

In [59]:
# 1. Carica l’immagine e normalizza in [0,1]
img = Image.open('cat.jpg').convert('RGB')
img_arr = np.asarray(img, dtype=np.float32) / 255.0


b = 0.17
a = -b
noisy_arr = add_salt_pepper_noise(img_arr)
noisy_img = Image.fromarray((noisy_arr * 255).astype(np.uint8))
# 4a. Salva su disco
#noisy_img.save('noisy_image.jpg')

# 4b. Oppure mostra a video (apre il viewer predefinito)
noisy_img.show()
#Andiamo

In [63]:
def add_salt_pepper_noise(image: np.ndarray, amount: float = 0.05, salt_vs_pepper: float = 0.5) -> np.ndarray:
    """
    :param image: Input image as float32 numpy array in [0,1], shape (H, W, 3).
    :param amount: Fraction of total pixels to corrupt.
    :param salt_vs_pepper: Fraction of corrupted pixels that are salt (vs pepper).
    :return: Noisy image as numpy array.
    """
    noisy = np.copy(image)
    ROWS, COLS, _ = image.shape
    num_pixels = ROWS * COLS
    num_salt = int(np.ceil(amount * num_pixels * salt_vs_pepper))
    num_pepper = int(np.ceil(amount * num_pixels * (1 - salt_vs_pepper)))

    # SALT
    coords_y = np.random.randint(0, ROWS, num_salt)
    coords_x = np.random.randint(0, COLS, num_salt)
    noisy[coords_y, coords_x, :] = 1.0

    # PEPPER
    coords_y = np.random.randint(0, ROWS, num_pepper)
    coords_x = np.random.randint(0, COLS, num_pepper)
    noisy[coords_y, coords_x, :] = 0.0

    return noisy

rng = np.random.default_rng()


def add_gaussian_noise(image: np.ndarray, mean=0.0, var=0.01):
    """
    :param image: Input image as a float32 numpy array with values in [0, 1].
    :param mean: Mean of Gaussian noise.
    :param var: Variance of Gaussian noise.
    :return: Noisy image as numpy array.
    """
    sigma = np.float32(var**0.5)
    noisy = image.copy()  # float32
    rng.standard_normal(size=image.shape, dtype=np.float32, out=noisy)
    noisy *= sigma
    noisy += image + np.float32(mean)
    np.clip(noisy, 0.0, 1.0, out=noisy)
    return noisy


def add_uniform_noise(image: np.ndarray, low=-0.1732, high=0.1732):
    """
    :param image: Input image as a float32 numpy array with values in [0, 1].
    :param low: Lower bound of uniform noise.
    :param high: Upper bound of uniform noise.
    :return: Noisy image as numpy array.
    """
    noisy = image.copy()  # float32
    rng.random(size=image.shape, dtype=np.float32, out=noisy)
    noisy *= np.float32(high - low)
    noisy += np.float32(low) + image
    np.clip(noisy, 0.0, 1.0, out=noisy)
    return noisy



def add_erlang_noise(image: np.ndarray, k=3, lam=17.32):
    """
    Add Erlang (Gamma) noise to an image, using shape k and rate λ.
    The Erlang PDF is: f(x) = (λ^k * x^(k-1) / (k-1)!) * e^{-λ x},  x >= 0

    :param image:     Input image as float32 numpy array in [0,1], shape (H, W, C).
    :param k:         Shape parameter (integer ≥ 1).
    :param lam:       Rate parameter λ (> 0).
    :return:          Noisy image as numpy array, clipped to [0,1].
    """
    theta = np.float32(1.0/lam)
    noisy = image.copy() # float32
    noise64 = rng.gamma(shape=k, scale=theta, size=image.shape) # genero rumore in float64 (per forza)
    noise = noise64.astype(np.float32) # cast to float32 (necessario)
    noise -= (np.float32(k) * theta)
    noisy += noise
    np.clip(noisy, 0.0, 1.0, out=noisy)
    return noisy

In [61]:
# 1. Carica e normalizza
img = Image.open('cat.jpg').convert('RGB')
arr = np.asarray(img, dtype=np.float32) / 255.0

noisy_arr = add_erlang_noise(arr, k=3, lam=25)
# 12,25
# 3. Riconverti e salva
noisy_img = Image.fromarray((noisy_arr * 255).astype(np.uint8))
noisy_img.show()
#noisy_img.save('path/to/noisy_erlang.jpg')

Bisogna calcolare la varianza per ogni rumore, e settare i parametri affinchè la varianza media sia 0.01.

In [64]:
def generate_dataset(input_dir: str, output_dir: str, batch_size: int = 32) -> None:
    """
    Genera un dataset con:
      - originali
      - salt & pepper (amount random in [0.01,0.10])
      - gaussian    (var random in [0.005,0.02], mean=0)
      - uniform     (r random in [0.10,0.20], noise in [-r,+r])
      - erlang      (k=3, λ random in [12,25])
    usando:
      • cv2.imread / cv2.imwrite per I/O veloce
      • caricamento a batch di batch_size immagini
    Alla fine salva anche labels.csv.
    """
    # Range dei parametri
    sp_range     = (0.01, 0.10)
    gauss_range  = (0.005, 0.02)
    uni_range    = (0.10, 0.20)
    erlang_range = (12.0, 25.0)
    k_erlang     = 3

    # Mappatura label e creazione cartelle
    variants = ['original','salt_pepper','gaussian','uniform','erlang']
    label_map = {
        'original':    'Original',
        'salt_pepper': 'Salt & Pepper',
        'gaussian':    'Gaussian',
        'uniform':     'Uniform',
        'erlang':      'Erlang'
    }
    for v in variants:
        os.makedirs(os.path.join(output_dir, v), exist_ok=True)

    # Lista di tutti i file immagine
    files = [f for f in os.listdir(input_dir)
             if f.lower().endswith(('.png','.jpg','.jpeg','.bmp','.tiff'))]

    records = []
    # Elaborazione a batch
    for i in range(0, len(files), batch_size):
        batch = files[i:i+batch_size]
        imgs = []
        # 1) CARICAMENTO BATCH in memoria con cv2
        for fn in batch:
            src = os.path.join(input_dir, fn)
            bgr = cv2.imread(src, cv2.IMREAD_COLOR)         # BGR uint8
            rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)      # RGB uint8
            imgs.append(rgb.astype(np.float32) / 255.0)     # float32 [0,1]

        # 2) PROCESSING per immagine nel batch
        for idx, fn in enumerate(batch):
            img = imgs[idx]  # float32 array (H,W,3)

            # — Originale —
            orig_uint8 = (img * 255).astype(np.uint8)
            orig_bgr   = cv2.cvtColor(orig_uint8, cv2.COLOR_RGB2BGR)
            out_orig   = os.path.join(output_dir, 'original', fn)
            cv2.imwrite(out_orig, orig_bgr)
            records.append({'filepath': out_orig, 'label': label_map['original']})

            # — Salt & Pepper —
            amt    = random.uniform(*sp_range)
            sp = add_salt_pepper_noise(img, amount=amt)
            sp_uint8 = (sp * 255).astype(np.uint8)
            sp_bgr   = cv2.cvtColor(sp_uint8, cv2.COLOR_RGB2BGR)
            out_sp   = os.path.join(output_dir, 'salt_pepper', fn)
            cv2.imwrite(out_sp, sp_bgr)
            records.append({'filepath': out_sp, 'label': label_map['salt_pepper']})

            # — Gaussian —
            var    = random.uniform(*gauss_range)
            g = add_gaussian_noise(img, var=var)
            g_uint8 = (g * 255).astype(np.uint8)
            g_bgr   = cv2.cvtColor(g_uint8, cv2.COLOR_RGB2BGR)
            out_g   = os.path.join(output_dir, 'gaussian', fn)
            cv2.imwrite(out_g, g_bgr)
            records.append({'filepath': out_g, 'label': label_map['gaussian']})

            # — Uniform —
            r      = random.uniform(*uni_range)
            u = add_uniform_noise(img, low=-r, high=+r)
            u_uint8 = (u * 255).astype(np.uint8)
            u_bgr   = cv2.cvtColor(u_uint8, cv2.COLOR_RGB2BGR)
            out_u   = os.path.join(output_dir, 'uniform', fn)
            cv2.imwrite(out_u, u_bgr)
            records.append({'filepath': out_u, 'label': label_map['uniform']})

            # — Erlang (Gamma) —
            lam    = random.uniform(*erlang_range)
            e = add_erlang_noise(img, k= k_erlang, lam=lam)
            e_uint8 = (e * 255).astype(np.uint8)
            e_bgr   = cv2.cvtColor(e_uint8, cv2.COLOR_RGB2BGR)
            out_e   = os.path.join(output_dir, 'erlang', fn)
            cv2.imwrite(out_e, e_bgr)
            records.append({'filepath': out_e, 'label': label_map['erlang']})

    # 3) Salvataggio CSV con tutte le label
    df = pd.DataFrame(records)
    df.to_csv(os.path.join(output_dir, 'labels.csv'), index=False)
    print(f"Dataset e labels salvati in '{output_dir}'")

In [66]:
# ESEGUO
input_directory = "CocoMini_25k/."
output_directory = "Dataset_25000x5."
generate_dataset(input_directory, output_directory)

KeyboardInterrupt: 

In [65]:
import time

# Benchmark
img = Image.open('cat.jpg').convert('RGB')
image = np.asarray(img, dtype=np.float32) / 255.0
functions = {
    'Salt & Pepper': lambda img: add_salt_pepper_noise(img),
    'Gaussian': lambda img: add_gaussian_noise(img),
    'Uniform': lambda img: add_uniform_noise(img),
    'Erlang (Gamma)': lambda img: add_erlang_noise(img)
}

N = 60
results = {}
for name, fn in functions.items():
    start = time.time()
    for _ in range(N):
        _ = fn(image)
    end = time.time()
    results[name] = end - start

for name, total in results.items():
    print(f"{name}: {total:.4f}s total, {total/N:.6f}s per run")


Salt & Pepper: 0.5151s total, 0.008586s per run
Gaussian: 2.7720s total, 0.046199s per run
Uniform: 1.2465s total, 0.020776s per run
Erlang (Gamma): 5.8798s total, 0.097997s per run
